In [2]:
import pandas as pd
import os
from pathlib import Path
import numpy as np

# Prepare input data for Mosna analysis

Mosna require an input file with cells coordinates, marker levels (expression as counts or protein levels) and both a column with cell type for each cell and a set of columns one hot encoded for celltypes

In IMMUCAN, IMC files for NSCLC2 are composed of two separate files:
- raw count files with protein levels 
- cell metadata files with celltypes, coordinates and lot more.

Moreover, Mosna suppose to start with one sigle file with all the samples and patients inside. In later steps, this initial file will be processed into single nodes and edges files with .parquet extention

In [3]:
raw_counts_path = Path("../../IMMUCAN_data/NSCLC2/01_Imaging/IMC/01_raw_counts")
cell_metadata_path = Path("../../IMMUCAN_data/NSCLC2/01_Imaging/IMC/01_cell_metadata")

output_folder = Path("../../IMMUCAN_data/NSCLC2/01_Imaging/IMC/02_combined_files")

In [4]:
raw_files = os.listdir(raw_counts_path)
metadata_files = os.listdir(cell_metadata_path)

In [11]:
# IDs of raw and metadata files stored in the file name  
raw_ids = [s.split('_#_raw_counts')[0] for s in raw_files]
raw_ids.sort()
print(raw_ids)

metadata_ids = [s.split('_#_cell_metadata')[0] for s in metadata_files]
metadata_ids.sort()
print(metadata_ids)

# Ideally, these ids should be the same
print(raw_ids == metadata_ids)

# Since we have checked they are equal, we can merge files based on their name
IMCs_all_TLS = [] # empty list to store all the final combined dfs
IMCs_all_otherROIs = [] # empty list to store all the final combined dfs

marker_columns = pd.read_csv(raw_counts_path/raw_files[0], index_col=0).columns.tolist() # Required to keep only relevant columns for Mosna in the final step
index_cd4 = marker_columns.index('CD4')
index_hladr = marker_columns.index('HLADR')

# Modify selected indices (later these raw colmns will be renamed to avoid duplicated columns in the hot-encoded columns creation step) 
marker_columns[index_cd4] = 'CD4_marker_col'
marker_columns[index_hladr] = 'HLADR_marker_col'

relevant_metadata_columns = ["Pos_X", "Pos_Y", "celltypes", "ROI", "tumor_patches"]
columns_to_select = relevant_metadata_columns + marker_columns

ROI_TLS = 4 # Set ROI_TLS to the ROI number that contains TLS

for raw_file in raw_files:
    # Attempt to read the raw data file, handle exceptions if file read fails
    try:
        df_raw = pd.read_csv(raw_counts_path/raw_file, index_col=0)  # First column is used as index for both dataframes
        df_raw.rename(columns={"CD4":"CD4_marker_col", "HLADR":"HLADR_marker_col"} , inplace=True) # To avoid duplicated columns during the hot-encoded columns creation   
    except Exception as e:
        print(f"Error reading {raw_file}: {e}")
        #continue
    
    
    # Extract the sample ID from the filename
    sample_id = raw_file.split('_#_raw_counts')[0]
    print("raw:", raw_file, "\n        Corresponding sample:", sample_id)
    
    # Filter metadata files to find the one starting with the sample_id
    matching_files = [f for f in metadata_files if f.startswith(sample_id)]
    
    if len(matching_files) == 1:
        metadata_file = matching_files[0]
        
        # Attempt to read the metadata file, handle exceptions if file read fails
        try:
            df_metadata = pd.read_csv(cell_metadata_path/metadata_file, index_col=0)
        except Exception as e:
            print(f"Error reading metadata file {metadata_file}: {e}")
            #continue
        
        print(" Corresponding metadata file:", metadata_file)

        # Further level of check: merge only if the number of rows (cells) matches
        if df_raw.shape[0] == df_metadata.shape[0]:
            try:
                df_combined = pd.merge(df_raw, df_metadata, left_index=True, right_index=True)
                file_name = sample_id + "_combined_raw_and_metadata.csv"
                df_combined.to_csv(output_folder / file_name)
                
                df_copy = df_combined[columns_to_select].copy()
                df_forMosna = pd.get_dummies(df_copy, columns=['celltypes'], drop_first=False, prefix=None) # one hot encoded cell types, as required by Mosna. Columns mustn't have any prefix in their name
                
                # Add back celltypes column used for the last step 
                if 'celltypes' not in df_forMosna.columns:
                    df_forMosna['celltypes'] = df_copy['celltypes']

                # Add some useful columns    
                df_forMosna['filename'] = file_name
                df_forMosna['sample'] = df_forMosna['filename'].str.split('-IMC').str[0]
                df_forMosna['patient'] = df_forMosna['filename'].str.split('-FIXT').str[0]    

                df_forMosna_TLS = df_forMosna[df_forMosna["ROI"] == ROI_TLS] # subset the df to analyze separatly the different ROIs
                df_forMosna_otherROIs = df_forMosna[df_forMosna["ROI"] != ROI_TLS]
                
                IMCs_all_TLS.append(df_forMosna_TLS)
                IMCs_all_otherROIs.append(df_forMosna_otherROIs)

                
            except Exception as e:
                print(f"Error during merging or saving the file: {e}")
    else:
        if len(matching_files) == 0:
            print(f"No matching metadata file found for {sample_id}")
        else:
            print(f"Multiple matching metadata files found for {sample_id}, please check data.")

# Here we concatenate in a single file all the dfs stored in the list "IMCs_all.*" 
IMC_all_TLS = pd.concat(IMCs_all_TLS)
IMC_all_TLS.to_parquet(output_folder / "IMC_all_TLS.parquet") # save in parquet format to store efficiently big amount of data 

IMC_all_otherROIs = pd.concat(IMCs_all_otherROIs)
IMC_all_otherROIs.to_parquet(output_folder / "IMC_all_otherROIs.parquet") 
    

['LUNG-NSCLC2-0552-FIXT-01-IMC1-01_#_IMMUcan_panel_1_1.10', 'LUNG-NSCLC2-0553-FIXT-01-IMC1-01_#_IMMUcan_panel_1_1.10', 'LUNG-NSCLC2-0554-FIXT-01-IMC1-01_#_IMMUcan_panel_1_1.10', 'LUNG-NSCLC2-0555-FIXT-01-IMC1-01_#_IMMUcan_panel_1_1.10', 'LUNG-NSCLC2-0556-FIXT-01-IMC1-01_#_IMMUcan_panel_1_1.10', 'LUNG-NSCLC2-0557-FIXT-01-IMC1-01_#_IMMUcan_panel_1_1.10', 'LUNG-NSCLC2-0558-FIXT-01-IMC1-01_#_IMMUcan_panel_1_1.10', 'LUNG-NSCLC2-0559-FIXT-01-IMC1-01_#_IMMUcan_panel_1_1.10', 'LUNG-NSCLC2-0560-FIXT-01-IMC1-01_#_IMMUcan_panel_1_1.10', 'LUNG-NSCLC2-0561-FIXT-01-IMC1-01_#_IMMUcan_panel_1_1.10', 'LUNG-NSCLC2-0562-FIXT-01-IMC1-01_#_IMMUcan_panel_1_1.10', 'LUNG-NSCLC2-0563-FIXT-01-IMC1-01_#_IMMUcan_panel_1_1.10', 'LUNG-NSCLC2-0564-FIXT-01-IMC1-01_#_IMMUcan_panel_1_1.10', 'LUNG-NSCLC2-0565-FIXT-01-IMC1-01_#_IMMUcan_panel_1_1.10', 'LUNG-NSCLC2-0566-FIXT-01-IMC1-01_#_IMMUcan_panel_1_1.10', 'LUNG-NSCLC2-0567-FIXT-01-IMC1-01_#_IMMUcan_panel_1_1.10', 'LUNG-NSCLC2-0568-FIXT-01-IMC1-01_#_IMMUcan_panel_1_1.1

/home/francesco.massaini/.local/lib/python3.10/site-packages/pandas/core/indexes/base.py:7834: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)


raw: LUNG-NSCLC2-0733-FIXT-01-IMC1-01_#_IMMUcan_panel_1_1.10_#_raw_counts_#_53a07871995741590ee6aff841d97936.csv 
        Corresponding sample: LUNG-NSCLC2-0733-FIXT-01-IMC1-01_#_IMMUcan_panel_1_1.10
 Corresponding metadata file: LUNG-NSCLC2-0733-FIXT-01-IMC1-01_#_IMMUcan_panel_1_1.10_#_cell_metadata_#_76c90531788f4a2cd39222ab51570a62.csv


/home/francesco.massaini/.local/lib/python3.10/site-packages/pandas/core/indexes/base.py:7834: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)


raw: LUNG-NSCLC2-0662-FIXT-01-IMC1-01_#_IMMUcan_panel_1_1.10_#_raw_counts_#_e3997182047c5f68467f1ce1957b7a6e.csv 
        Corresponding sample: LUNG-NSCLC2-0662-FIXT-01-IMC1-01_#_IMMUcan_panel_1_1.10
 Corresponding metadata file: LUNG-NSCLC2-0662-FIXT-01-IMC1-01_#_IMMUcan_panel_1_1.10_#_cell_metadata_#_3bedd3d378c66b3e36f054111b48220c.csv


...

/home/francesco.massaini/.local/lib/python3.10/site-packages/pandas/core/indexes/base.py:7834: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)


raw: LUNG-NSCLC2-0656-FIXT-01-IMC1-01_#_IMMUcan_panel_1_1.10_#_raw_counts_#_8310935ed3d6ea8096d644b339b14264.csv 
        Corresponding sample: LUNG-NSCLC2-0656-FIXT-01-IMC1-01_#_IMMUcan_panel_1_1.10
 Corresponding metadata file: LUNG-NSCLC2-0656-FIXT-01-IMC1-01_#_IMMUcan_panel_1_1.10_#_cell_metadata_#_7d0f9222bd162b8844e1f4a80798d2e8.csv


/home/francesco.massaini/.local/lib/python3.10/site-packages/pandas/core/indexes/base.py:7834: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)


raw: LUNG-NSCLC2-0651-FIXT-01-IMC1-01_#_IMMUcan_panel_1_1.10_#_raw_counts_#_5d89870728be4495404a8c7de269bd0c.csv 
        Corresponding sample: LUNG-NSCLC2-0651-FIXT-01-IMC1-01_#_IMMUcan_panel_1_1.10
 Corresponding metadata file: LUNG-NSCLC2-0651-FIXT-01-IMC1-01_#_IMMUcan_panel_1_1.10_#_cell_metadata_#_1f0aba382a3530bf6c961fa1a0c17489.csv


/home/francesco.massaini/.local/lib/python3.10/site-packages/pandas/core/indexes/base.py:7834: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)
